In [2]:
# doccano.json einlesen
import json
import spacy
from spacy.tokens import DocBin

# ----------------------------------
# ------------ Options: ------------
# ----------------------------------

#exportiert die entstandenen Daten als jsonl
doExport = False
# zeigt im searchText Bereich auch den Kontext an, in dem die gesuchten Begriffe stehen
showContext = True #False
# printet Labelanzahlen 
printCounter = False
# maximale Anzahl von Wörtern um die die Spangrenze (Ende) nach rechts verschoben werden kann
maxCounter = 30    

searchTexts = [
        "TextToSearch",
    ] 


# -----------------------------------
# ----------- Funktionen: -----------
# -----------------------------------

def importData():
    cvList = []
    with open("./Labelingstudie/Ergebnisse/data.jsonl", 'r') as f:
        for cv in f:
            cvList.append(json.loads(cv))  
    return cvList
  
def exportData(dicts, modelNumber, cvDicts):
    try:
        for cv in cvDicts:
            spacy.training.offsets_to_biluo_tags(nlp.make_doc(cv['text']), cv['label'])
        fileName = './Labelingstudie/Ergebnisse/editedData.jsonl'.format(modelNumber)
        with open(fileName, 'w', encoding='utf-8') as f:
            json.dump(dicts, f, ensure_ascii=False)
    except:
        print("ERROR WHILE EXPORTING")
        

"""
Benötigt um Span-/Labelgrenzen anzupassen, sodass sie den spaCy-Standards entsprechen.
Wird das nicht gemacht, kann es später zu Fehlern kommen.
Möglicherweise fallen hierdurch Label weg. 
Bspw. kann "Java-Programmierung" in spaCy nur als ganzes gelabelt werden.
"""
def adjustSpanBoundaries(start, end, label, maxCounter):
    span = doc.char_span(start, end, label=label)
    boundaryError = False

    newEnd = end
    counter = 0;
    #verschiebt die Grenze nach hinten, solange der span ungültig ist
    while span == None and not counter > maxCounter:
        counter = counter + 1
        newEnd = newEnd + 1
        span = doc.char_span(start, newEnd, label=label)

    # geht hier rein, falls der span immer noch ungültig ist
    if span == None:
        newEnd = end
        counter = 0;
        # setzt die Spangrenze zurück und verschiebt sie diesmal nach vorne, solange der span ungültig ist
        while span == None and not counter > maxCounter:
            counter = counter + 1
            start = start - 1
            span = doc.char_span(start, end, label=label)
        # gibt Fehler zurück sollte der span immer noch ungültig sein
        if span == None:
            boundaryError = True

    return boundaryError, span, start, newEnd 
                
                        
"""
Prüft, ob der aktuelle Span sich mit keinem anderen überlappt, da spaCy damit nicht umgehen kann.
Gibt einen entsprechenden boolean zurück.
"""      
def checkForOverlap(spanStart, spanEnd, labels):
    foundOverlap = False
    for otherStart, otherEnd, _ in labels:
        if otherStart == spanStart and otherEnd == spanEnd:
            foundOverlap = True
        elif spanStart < otherEnd and spanEnd > otherStart:
            foundOverlap = True
    return foundOverlap

"""
Printet alle gefundenen Spans in denen einer der übergebenen searchTexts vorkommt
"""
def searchForTokens(searchTexts, span, allreadyFound, showContext, cvNum, foundSearchedToken):
    spanText = span.text
    for searchText in searchTexts: 
        if searchText in spanText.lower():
            allreadyFound.append("{} - LABEL: {}".format(spanText, span.label_))
            if showContext:
                print("---- INCLUDES", searchText,": ", span.text, "; LABEL: ", label, " ----")
                contextLen = 1
                if (span.start - contextLen >= 0):
                    span.start = span.start - contextLen
                else:
                    span.start = 0                           
                if (span.end + contextLen <= len(doc)):
                    span.end = span.end + contextLen
                else:
                    span.end = len(doc)
                print("    ---- CONTEXT: ", span.text, " ----")
                foundSearchedToken.append({"text": spanText, "label": span.label_, "context": span.text})
                span.start = span.start + contextLen
                span.end = span.end - contextLen  
            else:
                print("---- INCLUDES", searchText,": ", span.text, "\n       LABEL: ", label, " ----") 
                print("         in CV:",cvNum)


# -----------------------------------
# -------------- Main: --------------
# -----------------------------------

# leeres neues Modell erstellen
nlp = spacy.blank("de")
cvList = importData()

# Arrays mit allen vergebenen Labeln
activities = []
skills = []
rollen = []
branchen = []

allreadyFound = []
# zählt wie oft die einzelnen Label vorkommen
labelCounter = []
foundSearchedToken = []

# speichert die CVs mit ihren Labels
cvDicts = []
dicts = {"CVs": cvDicts}
i=0

# geht jeden CV durch und überprüft alle Label in ihm
for cv in cvList:
    text = cv['text']
    annotations = cv['label']
    
    doc = nlp(text)
    
    counterSkills = 0
    counterActivities = 0
    counterRolle = 0
    counterBranche = 0
    
    # speichert alle Label in dictionary
    labels = []
    cvDict = {
      "label": labels
    }
    
    # geht alle Label durch
    for annotation in annotations:
        label = annotation[2]
        boundaryError, span, spanStart, spanEnd = adjustSpanBoundaries(int(annotation[0]), int(annotation[1]), label, maxCounter)
            
        if not boundaryError:
            # war durch einen Fehler des Annotation Tools in den Daten, obwohl es nicht gelabelt wurde
            if "aufgabenschwerpunkte" in span.text.lower():
                continue
            searchForTokens(searchTexts, span, allreadyFound, showContext, i, foundSearchedToken)
            
            # sortiert sich überlappende Label aus  
            foundOverlap = checkForOverlap(spanStart, spanEnd, labels)
                
            # speichert Label, sofern es sich mit keinem anderen überschneidet
            if not foundOverlap:
                labels.append([spanStart, spanEnd, label])
                # zählt counter hoch
                if label == "Tätigkeit":
                    activities.append(span.text)
                    counterActivities = counterActivities + 1
                elif label == "Skill":
                    skills.append(span.text)  
                    counterSkills = counterSkills + 1
                elif label == "Branche": 
                    counterBranche = counterBranche + 1
                    branchen.append(span.text)
                elif label == "Rolle": 
                    counterRolle = counterRolle + 1
                    rollen.append(span.text)
                
    cvDicts.append(cvDict)
    labelCounter.append({
        "Tätigkeiten": counterActivities, 
        "Skills": counterSkills, 
        "Branchen": counterBranche, 
        "Rollen": counterRolle, 
        "Gesamt": (counterActivities+counterSkills+counterBranche+counterRolle)
    })
    i = i+1

# Printet wie häufig die Label von den einzelnen Probanden verteilt wurden
for element in labelCounter:
    print(element)


{'Tätigkeiten': 43, 'Skills': 30, 'Branchen': 3, 'Rollen': 14, 'Gesamt': 90}
{'Tätigkeiten': 44, 'Skills': 20, 'Branchen': 6, 'Rollen': 8, 'Gesamt': 78}
{'Tätigkeiten': 40, 'Skills': 63, 'Branchen': 6, 'Rollen': 12, 'Gesamt': 121}
{'Tätigkeiten': 13, 'Skills': 13, 'Branchen': 7, 'Rollen': 8, 'Gesamt': 41}
{'Tätigkeiten': 57, 'Skills': 31, 'Branchen': 8, 'Rollen': 10, 'Gesamt': 106}
{'Tätigkeiten': 46, 'Skills': 28, 'Branchen': 4, 'Rollen': 14, 'Gesamt': 92}
{'Tätigkeiten': 45, 'Skills': 38, 'Branchen': 5, 'Rollen': 15, 'Gesamt': 103}
{'Tätigkeiten': 18, 'Skills': 33, 'Branchen': 5, 'Rollen': 24, 'Gesamt': 80}
{'Tätigkeiten': 39, 'Skills': 31, 'Branchen': 6, 'Rollen': 10, 'Gesamt': 86}
{'Tätigkeiten': 12, 'Skills': 15, 'Branchen': 1, 'Rollen': 6, 'Gesamt': 34}
{'Tätigkeiten': 36, 'Skills': 41, 'Branchen': 15, 'Rollen': 12, 'Gesamt': 104}
{'Tätigkeiten': 38, 'Skills': 23, 'Branchen': 5, 'Rollen': 10, 'Gesamt': 76}


In [3]:
# printet die Ergebnisse
import distance 

# ----- Options: -----
printDoubles = False #True 
printAllLabels = False


# damit jeder Skill nur einmal in der Liste ist
skills.sort()
activities.sort()
rollen.sort()
branchen.sort()

#Erstellt dicts, um zu zählen, wie oft jede Wortsequenz gelabelt wurde
setList = list(set(skills)) 
skillDict = {i:skills.count(i) for i in setList}
setList = list(set(activities)) 
activitiesDict = {i:activities.count(i) for i in setList}
setList = list(set(rollen)) 
rollenDict = {i:rollen.count(i) for i in setList}
setList = list(set(branchen)) 
branchenDict = {i:branchen.count(i) for i in setList}
skillsWithCount = []
activitiesWithCount = []
rollenWithCount = []
branchenWithCount = []

for entry in skillDict:
    skillsWithCount.append("{} - ({}x)".format(entry, skillDict.get(entry)))  
for entry in activitiesDict:
    activitiesWithCount.append("{} - ({}x)".format(entry, activitiesDict.get(entry)))   
for entry in rollenDict:
    rollenWithCount.append("{} - ({}x)".format(entry, rollenDict.get(entry))) 
for entry in branchenDict:
    branchenWithCount.append("{} - ({}x)".format(entry, branchenDict.get(entry))) 

#Sorgt dafür, dass jeder Eintrag nur einmal in den Listen vorkommt
activities = list(dict.fromkeys(activities))
skills = list(dict.fromkeys(skills)) 
rollen = list(dict.fromkeys(rollen)) 
branchen = list(dict.fromkeys(branchen)) 
    
skillsWithCount.sort()
activitiesWithCount.sort()
rollenWithCount.sort()
branchenWithCount.sort()

print("Anz unique Skills: ", len(skillsWithCount))
print("Anz unique Activities: ", len(activitiesWithCount))
print("Anz unique Branchen: ", len(branchenWithCount))
print("Anz unique Rollen: ", len(rollenWithCount))

if printDoubles:
    print("\n=> Kommt als Skill und als Tätigkeit vor: ")
    for skill in skills:
        for activity in activities:
            if skill == activity:
                print (skill)
            elif abs(len(activity) - len(skill)) < 3:
                maxLevDist = int(len(activity)/5)
                if distance.levenshtein(skill, activity) <= maxLevDist:
                    print("Similar: ", skill, " -- ", activity)

if printAllLabels:
    print("\n ---------------------------------------------------------------------  \n")
    print("=> Skills: ")
    for skill in skillsWithCount:
        print(skill)

    print("\n ---------------------------------------------------------------------  \n")
    print("=> Tätigkeiten: ")
    for activity in activitiesWithCount:
        print(activity)

    print("\n ---------------------------------------------------------------------  \n")
    print("=> Rollen: ")
    for rolle in rollenWithCount:
        print(rolle)

    print("\n ---------------------------------------------------------------------  \n")
    print("=> Branchen: ")
    for branche in branchenWithCount:
        print(branche)


Anz unique Skills:  118
Anz unique Activities:  110
Anz unique Branchen:  27
Anz unique Rollen:  53


In [6]:
# Printet alle gelabelten Wortfolgen und wie häufig sie jedes Label bekommen haben
#  Kann nicht unterscheiden, ob eine Wortfolge in mehreren Kontexten gelabelt wurde

# ---- Options: ---- 
exportFiles = False


allLabels = {}

for skill in skillDict:
    allLabels[skill] = {"skiCount":skillDict.get(skill), "actCount":0, "braCount":0, "rolCount":0}

for activity in activitiesDict:
    allreadyIn = False
    #print(activity)
    if activity in allLabels.keys():  
        #print("found!")
        allLabels[activity]["actCount"] = activitiesDict.get(activity)
        allreadyIn = True
    if not allreadyIn:
        #print("enter!")
        allLabels[activity] = {"skiCount":0, "actCount":activitiesDict.get(activity), "braCount":0, "rolCount":0}
           
for branche in branchenDict:
    allreadyIn = False
    if branche in allLabels.keys():  
        #print("found!")
        allLabels[branche]["braCount"] = branchenDict.get(branche)
        allreadyIn = True
    if not allreadyIn:
        allLabels[branche] = {"skiCount":0, "actCount":0, "braCount":branchenDict.get(branche), "rolCount":0}
    
for rolle in rollenDict:
    allreadyIn = False
    if rolle in allLabels.keys():  
        #print("found!")
        allLabels[rolle]["rolCount"] = rollenDict.get(rolle)
        allreadyIn = True
    if not allreadyIn:
        allLabels[rolle] = {"skiCount":0, "actCount":0, "braCount":0, "rolCount":rollenDict.get(rolle)}

print("Number of different Labels:", len(allLabels))
    
allLabelsNew = []
for key in sorted(allLabels):
    allLabelsNew.append({"label": key,"skiCount": allLabels[key]["skiCount"], "actCount": allLabels[key]["actCount"], "braCount": allLabels[key]["braCount"], "rolCount": allLabels[key]["rolCount"]})
    
for element in allLabelsNew:
    print(element) 

    
if exportFiles:        
    fileName = './Labelingstudie/Ergebnissübersicht.jsonl'
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(allLabels, f, ensure_ascii=False)
    fileName = './Labelingstudie/Ergebnissübersicht_withIndent.jsonl'
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(allLabelsNew, f, ensure_ascii=False, indent=3)

    import csv
    fileName = './Labelingstudie/Ergebnissübersicht.csv'
    with open(fileName, 'w') as f:  
        w = csv.DictWriter(f, allLabels.keys())
        w.writeheader()
        w.writerow(allLabels)



Number of different Labels: 238
{'label': '(BA/Testing)', 'skiCount': 0, 'actCount': 1, 'braCount': 0, 'rolCount': 0}
{'label': '1st, 2nd und 3rd Level Support', 'skiCount': 2, 'actCount': 3, 'braCount': 0, 'rolCount': 2}
{'label': '3rd Level Support', 'skiCount': 0, 'actCount': 0, 'braCount': 0, 'rolCount': 1}
{'label': 'ABAP', 'skiCount': 10, 'actCount': 0, 'braCount': 0, 'rolCount': 0}
{'label': 'ABAP-Programmierung', 'skiCount': 8, 'actCount': 1, 'braCount': 0, 'rolCount': 0}
{'label': 'Abstimmung', 'skiCount': 0, 'actCount': 3, 'braCount': 0, 'rolCount': 0}
{'label': 'Abstimmung mit Subsystemen', 'skiCount': 0, 'actCount': 4, 'braCount': 0, 'rolCount': 0}
{'label': 'Access Management', 'skiCount': 2, 'actCount': 2, 'braCount': 0, 'rolCount': 2}
{'label': 'Access Management und Controlling', 'skiCount': 0, 'actCount': 1, 'braCount': 0, 'rolCount': 0}
{'label': 'Agile Testing', 'skiCount': 7, 'actCount': 2, 'braCount': 0, 'rolCount': 0}
{'label': 'Analyse', 'skiCount': 0, 'actCount'